In [1]:
# entity

In [11]:
import os
os.getcwd()

'/Users/admin/Documents/Projects/Wafer-Fault-Detection/research'

In [12]:
os.chdir("../")

In [13]:
os.getcwd()

'/Users/admin/Documents/Projects/Wafer-Fault-Detection'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from src.wafer_fault_detection import constants

In [4]:
constants.CONFIG_FILE_PATH

PosixPath('config/config.yaml')

In [5]:
constants.PARAMS_FILE_PATH

PosixPath('params.yaml')

In [6]:
from wafer_fault_detection.utils import read_yaml, create_directories

In [21]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = constants.CONFIG_FILE_PATH,
        params_filepath = constants.PARAMS_FILE_PATH,):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [22]:
c = ConfigurationManager()
config = c.get_data_ingestion_config()
print(config)

[2023-11-15 20:36:04,823: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-11-15 20:36:04,831: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-15 20:36:04,839: INFO: common: created directory at: artifacts]
[2023-11-15 20:36:04,841: INFO: common: created directory at: artifacts/data_ingestion]
DataIngestionConfig(root_dir='artifacts/data_ingestion', source_URL='https://github.com/rizal-muhammed/datasets/raw/main/wafer-fault-data.zip', local_data_file='artifacts/data_ingestion/wafer-fault-data.zip', unzip_dir='artifacts/data_ingestion')


In [37]:
import os
import requests
import zipfile
from urllib.parse import urlparse

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    # def download_file(self):
    #     logger.info("Trying to download file...")
    #     if not os.path.exists(self.config.local_data_file):
    #         logger.info("Download started...")
    #         filename, headers = request.urlretrieve(
    #             url=self.config.source_URL,
    #             filename=self.config.local_data_file
    #         )
    #         logger.info(f"{filename} download! with following info: \n{headers}")
    #     else:
    #         logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")       
    
    def download_file(self):
        logger.info("Trying to download file...")
        if not os.path.exists(self.config.local_data_file):
            url = self.config.source_URL
            response = requests.get(url)

            if response.status_code == 200:
            # Extract the file name from the URL
                url_path = urlparse(url).path
                file_name = url_path.split("/")[-1]

                with open(file_name, "wb") as file:
                    file.write(response.content)

            print(f"Downloaded {file_name} successfully")

            # Unzip the downloaded file
            with zipfile.ZipFile(file_name, "r") as zip_ref:
                zip_ref.extractall(self.config.unzip_dir)
            
            print(f"Unzipped {file_name} successfully to 'extracted_folder'")

            os.remove(file_name)
                
        else:
            print(f"Failed to download. Status code: {response.status_code}")

In [38]:
d = DataIngestion(config)

In [39]:
d.download_file()

Downloaded wafer-fault-data.zip successfully
Unzipped wafer-fault-data.zip successfully to 'extracted_folder'
